In [1]:
import pandas as pd
import numpy as np
from cytoolz import *
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
df = pd.read_parquet('s3://ling583/project1.parquet', storage_options={'anon':True})

In [3]:
from reut_tokenizer import MWETokenizer

tokenizer = MWETokenizer(open('reuters-terms.txt')) #incorporating reut_tokenizer.py

In [4]:
df['tokens'] = pd.Series(df['text'].progress_apply(tokenizer.tokenize)) # processin

  0%|          | 0/50085 [00:00<?, ?it/s]

In [5]:
import tomotopy as tp
import time

In [6]:
k = 70 #determines shape of whole model
min_df = 30 #any word that doesn't occur in more than x reviews gets ignored, addresses low-frequency words
rm_top = 80 #currently set to top 75 most common words (meant to remove), like stopwords...can adjust to, say, top 100
tw = tp.TermWeight.ONE #everything is weighed the same @ 1, check other strategies
alpha = 0.1 #term distribution
eta = 0.01 #topic distribution, alpha and eta are probably fine. 830min mark
tol = 1e-2

#selecting parameters for topic modeling

In [7]:
%%time

mdl = tp.LDAModel(k=k, min_df=min_df, rm_top=rm_top, tw=tw, alpha=alpha, eta=eta)

for doc in df['tokens']:
    if doc:
        mdl.add_doc(doc)

#TRAINING LOOP here
last = np.NINF
for i in range(0, 5000, 50):
    mdl.train(50)
    ll = mdl.ll_per_word
    print(f'{i:5d} LL = {ll:7.4f}', flush=True)
    if ll - last < tol:
        break
    else:
        last = ll

print(f'Done!') #When difference between a result and the result immediately preceding it breaks tolerance hyperparam, DONE 

    0 LL = -8.0646
   50 LL = -7.9310
  100 LL = -7.8870
  150 LL = -7.8656
  200 LL = -7.8526
  250 LL = -7.8415
  300 LL = -7.8350
Done!
CPU times: user 9min 6s, sys: 2.98 s, total: 9min 9s
Wall time: 2min 47s


In [8]:
for k in range(mdl.k):
    print(f'{k:3d} ', ', '.join(s for s,_ in mdl.get_topic_words(k)))
    
# my different topics

  0  earnings, 1997, results, profit, quarter, operating, 1995, per, revenues, growth
  1  mexico, mexican, newsroom, telmex, portugal, de, escudos, baht, budapest, thai
  2  tax, should, bill, state, could, public, if, labor, under, law
  3  cents, ltd, gas, corp, energy, carnival, oil, ordinary, natural, holdings
  4  eu, countries, satellite, trade, united_states, european, launch, world, agreement, telecommunications
  5  uld, ldd, port, tons, loading, berthing, unloading, soybeans, sugar, bulk
  6  aircraft, boeing, airbus, fleet, planes, order, aerospace, jets, engines, delivery
  7  brazil, telebras, swissair, reais, brazilian, swiss, sabena, sao_paulo, state, francs
  8  canadian, canada, irish, toronto, reuters, air_canada, ote, bureau, athens, plan
  9  hong_kong, japan, says, japanese, cathay_pacific, airways, china, asia, cathay, tokyo
 10  plane, people, passengers, officials, airport, crew, when, police, flight, into
 11  strike, drivers, truckers, french, france, truck, 

In [9]:
import pyLDAvis #takes model we've built, converts it (by "rearranging" model) into a form that another tool can use/ingest

topic_term_dists = np.stack([mdl.get_topic_word_dist(k)
                             for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq
prepared_data = pyLDAvis.prepare(topic_term_dists,
                                 doc_topic_dists,
                                 doc_lengths,
                                 vocab,
                                 term_frequency, 
                                 mds='tsne', 
                                 sort_topics=False
                                 )

# ran on my own to check how this model is

/opt/conda/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


In [10]:
pyLDAvis.display(prepared_data) # also paired with this to ensure model checks out geometrically

In [58]:
topics=pd.DataFrame({'words': [' '.join(map(first, mdl.get_topic_words(k))) for k in range(mdl.k)]})

In [59]:
topics.head() 

,words
0,earnings 1997 results profit quarter operating...
1,mexico mexican newsroom telmex portugal de esc...
2,tax should bill state could public if labor un...
3,cents ltd gas corp energy carnival oil ordinar...
4,eu countries satellite trade united_states eur...


In [60]:
topics.to_csv('topics.csv', index=False)

In [101]:
topics=pd.read_csv('topics.csv') #reading csv changes back in

In [102]:
topics #have created labels for topics through csv

,words,label
0,earnings 1997 results profit quarter operating...,stocks
1,mexico mexican newsroom telmex portugal de esc...,NaN
2,tax should bill state could public if labor un...,laws
3,cents ltd gas corp energy carnival oil ordinar...,NaN
4,eu countries satellite trade united_states eur...,NaN
...,...,...
65,day ads jakarta_newsroom volume high low repre...,NaN
66,prices rates fuel costs rate per charges price...,prices
67,port oil shipping ship ships ports tanker cont...,NaN
68,eurotunnel debt tunnel fire french shareholder...,NaN


In [103]:
topics.dropna() 

,words,label
0,earnings 1997 results profit quarter operating...,stocks
2,tax should bill state could public if labor un...,laws
6,aircraft boeing airbus fleet planes order aero...,aircrafts
9,hong_kong japan says japanese cathay_pacific a...,east asian
20,hong_kong ltd china 852 2843 hong_kong_newsroo...,hong kong
22,bank issue investors price capital offer inves...,banks
31,french francs france france_telecom air_france...,french
32,telecommunications network telephone telecom c...,communications
34,singapore malaysia ltd bhd indonesia ringgit s...,sijori
42,india indian rupees foreign bombay ltd state p...,south asian


In [104]:
df['text'].iloc[32] # the article

'The Sands Regent said Tuesday it filed a lawsuit against the Mississippi Port Authority and the Mississippi Department of Economic Development, because they refused to give Copa clearance to expand its casino operations in Gulfport, Mississippi. The suit also comes in response to recent notifications by the authority and the development group that Copa must vacate its casino site when its lease expires in October 1999. In its suit, filed in Circuit Court in Harrison County, Mississippi, Sands alleges failure to deal in good faith, breach of contract and misrepresentation.   Sands Regent said Copa had wanted to build a hotel near the site and bring in a casino barge to replace or supplement the current casino vessel. Copa is asking for monetary damages and declaratory relief giving them the right to build the hotel and bring in a new barge. The Port Authority has said it needs the location to expand shipping and related commercials activitie, according to Sands Regent.'

In [105]:
mdl.docs[32] #mdl model

<tomotopy.Document with words="the sands regent said tuesday it filed a lawsuit against the mississippi port_authority and the mississippi department of economic development because they refused to give copa clearance to expand its casino operations in gulfport mississippi the suit also comes in response to recent notifications by the authority and the development group that copa must vacate its casino site when its lease expires in october 1999 in its suit filed in circuit court in harrison county mississippi sands alleges failure to deal in good faith breach of contract and misrepresentation sands regent said copa had wanted to build a hotel near the site and bring in a casino barge to replace or supplement the current casino vessel copa is asking for monetary damages and declaratory relief giving them the right to build the hotel and bring in a new barge the port_authority has said it needs the location to expand shipping and related commercials activitie according to sands regent">

In [106]:
mdl.docs[32].get_topics() # topics and weights

[(41, 0.3881930708885193),
 (56, 0.20708681643009186),
 (23, 0.19411778450012207),
 (67, 0.09080846607685089),
 (2, 0.07830994576215744),
 (54, 0.014601534232497215),
 (14, 0.013291076757013798),
 (25, 0.0010119788348674774),
 (0, 0.0007511041476391256),
 (19, 0.000613199605140835)]

In [107]:
[t for t,w in mdl.docs[32].get_topics() if w>.01] #filtering for weight threshold, only topics

[41, 56, 23, 67, 2, 54, 14]

In [108]:
[topics['label'].loc[t] for t,w in mdl.docs[32].get_topics() if w>.01]

[nan, nan, nan, nan, 'laws', nan, nan]

In [109]:
set(filter(pd.notna, [topics['label'].loc[t] for t,w in mdl.docs[32].get_topics() if w>.01]))

{'laws'}

In [110]:
articles=[]
for i in range(0,len(mdl.docs)):
    articles.append(set(filter(pd.notna, [topics['label'].loc[t] for t,w in mdl.docs[i].get_topics() if w>.01])))

In [111]:
from collections import Counter
counter=Counter()

for k in articles:
    for label in k:
        counter[label] +=1
        
counter

#number of articles that fit the label

Counter({'banks': 8023,
         'stocks': 10776,
         'trading': 7296,
         'laws': 10760,
         'prices': 4492,
         'communications': 7213,
         'chinese': 1882,
         'sijori': 2720,
         'french': 2607,
         'south asian': 1487,
         'east asian': 2494,
         'aircrafts': 3338,
         'hong kong': 1972})